# **실습 시작전 런타임 환경 확인: 상단 탭 Runtime --> change runtime type --> Python3 T4GPU**

## **Session 1: transformers 라이브러리를 이용한 소형언어모델 호출 및 실행**

---

In [ ]:
!pip install transformers evaluate accelerate torch huggingface_hub

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import huggingface_hub

In [ ]:
# Llama, Gemma 등 접근 권한을 관리하는 모델의 경우 로그인 -> Token 생성 -> 접근 승인 과정이 필요
# huggingface_hub.login() #회원가입 > Setting 클릭 > Assess Tokens 클릭 > New Tokens 클릭

In [ ]:
model_id = "Qwen/Qwen2.5-1.5B-Instruct"

### **pipeline 모듈을 이용한 생성**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    dtype=torch.bfloat16,
    # torch_dtype=torch.float32,
    device="cuda"
)

messages = [
    {"role": "system", "content": "너는 사이버대학교의 진학상담 챗봇이다. 항상 한글로 상담사 처럼 대답해"},
    {"role": "user", "content": "너는 누구니?"},
]

outputs = pipe(messages)

print(outputs[0]["generated_text"][-1]['content'])

In [ ]:
outputs   # 코드 수준에서는 리스트 내부에 딕셔너리 형태로 생성된 텍스트가 저장

### **이전 대화 기록을 바탕으로한 생성**

In [ ]:
chat = outputs[0]["generated_text"]
chat.append({"role": "user", "content": "인공지능 분야 대학원 진학을 하고 싶은데 전망은 어때?"})
outputs = pipe(chat)
print(outputs[0]["generated_text"][-1]['content'])

### **GPU 메모리 점유율 확인**

In [ ]:
!nvidia-smi

### **원활한 실습을 위한 세션 재실행 (상단 탭 Runtime --> Restart session --> !nvidia-smi)**


---





### **소형언어모델의 메소드를 이용한 토크나이저 형태 및 생성 결과 확인**

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import huggingface_hub

model_id = "Qwen/Qwen2.5-1.5B-Instruct"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map="cpu") # CPU 환경에서 구동할경우 device_map="cpu"

tokenizer.save_pretrained("tokenizer_sample")

### **토크나이저의 출력값 확인(서브워드 토크나이저 특성)**

In [ ]:
# sequence = "너는 사이버대학교의 진학상담 챗봇이다. 항상 한글로 상담사 처럼 대답해" # 한글 예제
sequence = "model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map=cpu)" # 영어 및 코드 텍스트 예제
tokens = tokenizer.tokenize(sequence)

print(tokens) # 바이트 수준으로 구분된 토큰 확인

['model', 'Ġ=', 'ĠAuto', 'Model', 'For', 'C', 'ausal', 'LM', '.from', '_pre', 'trained', '(model', '_id', ',', 'Ġtrust', '_remote', '_code', '=True', ',', 'Ġdevice', '_map', '=', 'cpu', ')']


In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids) # 각 토큰은 인덱스 값을 가짐

[2528, 284, 8979, 1712, 2461, 34, 79074, 10994, 6387, 10442, 35722, 7635, 842, 11, 6950, 36425, 4136, 3618, 11, 3671, 5376, 28, 16475, 8]


In [ ]:
messages = [
    {"role": "system", "content": "너는 사이버대학교의 진학상담 챗봇이다. 항상 한글로 상담사 처럼 대답해"},
    {"role": "user", "content": "너는 누구니?"},
]
messages_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
messages_chat # 토크나이저에 입력되는 약속된 형태의 템플릿

In [ ]:
inputs = tokenizer(messages_chat, return_tensors="pt", add_special_tokens=False)
inputs = {key: tensor.to("cuda") for key, tensor in inputs.items()}
model.to("cuda")

In [ ]:
inputs # 실제 소형언어모델로 입력되는 형태 확인

### **모델의 메소드를 이용한 생성결과 확인**

In [ ]:
generation_config = {
            "max_new_tokens": 100,
            "num_beams": 4,
            "no_repeat_ngram_size": 2,
            "early_stopping": True,
            "do_sample": True,
            "temperature": 0.7,
            "pad_token_id": tokenizer.eos_token_id
        }

outputs = model.generate(**inputs,  **generation_config)
decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
print("Decoded output:\n", decoded_output)

In [ ]:
outputs[0] # 출력 결과: 토큰 인덱스

### **출력결과 시현을 위한 후처리 과정 이해**

In [ ]:
inputs['input_ids'].size(1)

In [ ]:
outputs[0][inputs['input_ids'].size(1):].size(0) # 모델 출력에서 입력값의 토큰 수만큼을 제외한 나머지만 시현

In [ ]:
!nvidia-smi # GPU 메모리 점유율 확인

### **원활한 실습을 위한 세션 재실행 (상단 탭 Runtime --> Restart session --> !nvidia-smi)**


---





## **Session 2: 올라마(Ollama) 라이브러리를 이용한 RAG 실습**

### **사전 준비: Ollama Installation & Execution(구 버전)**




In [ ]:
!pip install colab-xterm

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
%load_ext colabxterm

In [ ]:
%xterm # ollama serve &

In [ ]:
!ollama list

In [ ]:
!ollama --version

In [ ]:
!killall ollama

In [ ]:
!ollama pull aya:8b
# !ollama pull aya-expanse:8b

In [ ]:
%xterm # ollama run aya-expanse:8b

### **사전 준비: Ollama Installation & Execution(새 버전)**


In [ ]:
!sudo apt-get update && sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

!pip install ollama -q

### **Session 2-1: Ollama 기초 활용**

#### **ollama 환경 실행**

In [ ]:
import subprocess
import time
import ollama

subprocess.run(["pkill", "ollama"]) # 터미널 기준 >> pkill -9 ollama
subprocess.Popen(['ollama', 'serve']) # 터미널 기준 >> ollama serve
while True:
    try:
        ollama.list()
        print("\n✅ 서버 준비 완료! 이제 AI와 대화할 수 있습니다.")
        break
    except Exception:
        print(".", end="", flush=True)
        time.sleep(1)

In [ ]:
!ollama list # 활용 가능한 모델 종류 확인

In [ ]:
!ollama --version

In [ ]:
!nvidia-smi

#### **소형 언어모델 다운로드**

In [ ]:
# !ollama pull aya:8b
!ollama pull qwen2.5:1.5b

# ollama 에서 제공하는 모델 종류 참조 페이지: https://ollama.com/search

In [ ]:
!ollama list # 활용 가능한 모델 종류 확인

In [ ]:
!nvidia-smi

#### **ollama 메소드를 활용한 텍스트 생성**

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

# model_name = 'aya:8b'
model_name = 'qwen2.5:1.5b'

result_stream = ollama.chat(
      model=model_name,
      messages=messages,
      stream=True,
      # stream=False,
      # keep_alive=0
    )

for chunk in result_stream:
    content = chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
print('\n')

In [ ]:
# stream이 False 인 경우
contents_output = result_stream.message.content
contents_output

In [ ]:
!nvidia-smi # GPU 메모리 점유율 확인

### **Session 2-2: RAG 적용을 위한 전처리(pre-processing)**




#### **RAG 모듈 및 pdf 파일 처리를 위한 라이브러리 설치**

In [ ]:
!pip install langchain_community langchain-text-splitters langchain_huggingface sentence_transformers pdfplumber pypdf faiss-cpu -q

In [9]:
import pdfplumber
from collections import defaultdict
import itertools
import re
import os
import pandas as pd
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### **pdf 파일 로드 및 텍스트 구조 확인**

In [12]:
record_file = 'rag_example.pdf'

loader = PyPDFLoader(record_file)
documents = loader.load()
filtered_doc = documents[1:4]

In [ ]:
filtered_doc # pdf 파일 로드 후 텍스트 저장 변수 구조

#### **검색 효율성 향상을 위한 텍스트 청킹(chunking)**

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
chunking_data = text_splitter.split_documents(filtered_doc)

In [ ]:
chunking_data[0:4]

#### **표 데이터 추출 후 텍스트 청킹**

In [18]:
tables = []
with pdfplumber.open(record_file) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            df = pd.DataFrame(table[1:], columns=table[0])
            tables.append(df)

In [ ]:
len(tables) # 추출 된 테이블의 수

In [ ]:
tables[0] # 테이블 예시

#### **추출 된 표 중 특정 범위에 대해서만 Vector DB화**

In [22]:
filtered_tables = tables[7:21]
text_dict = defaultdict()
text = ""

In [23]:
# 텍스트 데이터 전처리(정규 표현식 활용)
for i, table in enumerate(filtered_tables):
    if isinstance(table, pd.DataFrame):
        table = table.ffill() # 데이터 채우기(이전 행의 값 그대로 사용)
        table = table.apply(lambda x: table.columns + ":" + x.astype(str), axis=1)
        tmp_text = table.to_csv(index=False, header=False)
        tmp_text = re.sub(r'\n"보 장', 'TEMP_REPLACE', tmp_text)
        tmp_text = re.sub(r'\n보 장', 'TEMP_2REPLACE', tmp_text)
        tmp_text = re.sub(r'\n', ' ', tmp_text)
        tmp_text = re.sub(r'TEMP_REPLACE', '\n"보 장', tmp_text)
        tmp_text = re.sub(r'TEMP_2REPLACE', '\n"보 장', tmp_text)
        text_dict[i] = tmp_text.split("\n")
        text += tmp_text

In [25]:
text_list = list(itertools.chain(*text_dict.values()))

In [ ]:
text_list[0] # 결과 확인

#### **Vector DB에 입력하기 위한 형식으로 변환**

In [29]:
chunking_data = [Document(page_content=page, metadata=dict(page=i)) for i, page in enumerate(text_list)]

In [ ]:
chunking_data # 결과 확인

### **Session 2-3: RAG 적용**

*   벡터 스토어 라이브러리: 벡터 유사도 검색(FAISS)
*   임베딩 모델: 문장을 임베딩 벡터로 투영



In [31]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

import os
from tqdm import tqdm
import ollama
import pandas as pd
import pdfplumber
from collections import defaultdict
import itertools
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#### **임베딩 모델 호출(SentenceTransformer 라이브러리 활용)**

In [ ]:
model_name = "BAAI/bge-m3"
if not os.path.exists(model_name):
    model = SentenceTransformer(model_name)
    model.save(model_name)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={
        'device': "cuda",
        'trust_remote_code': True,
    },
    encode_kwargs={'normalize_embeddings': True},
)

#### **문장의 임베딩 변환결과 예시**

In [ ]:
emb_vectors = embeddings.embed_documents([
    "안녕하세요.",
    "반갑습니다.",
    "감사합니다.",
])
print(emb_vectors[0][:4])
print(emb_vectors[1][:4])
print(emb_vectors[2][:4])

In [ ]:
len(emb_vectors[0])

#### **외부 참조 데이터 전처리(이전 세션 과정과 동일)**

In [37]:
chunking_data = ""

In [38]:
record_file = 'rag_example.pdf'

tables = []
with pdfplumber.open(record_file) as pdf:
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            df = pd.DataFrame(table[1:], columns=table[0])
            tables.append(df)

filtered_tables = tables[7:21]
text_dict = defaultdict()
text = ""

for i, table in enumerate(filtered_tables):
    if isinstance(table, pd.DataFrame):
        table = table.ffill() # 데이터 채우기(이전 행의 값 그대로 사용)
        table = table.apply(lambda x: table.columns + ":" + x.astype(str), axis=1)
        tmp_text = table.to_csv(index=False, header=False)
        tmp_text = re.sub(r'\n"보 장', 'TEMP_REPLACE', tmp_text)
        tmp_text = re.sub(r'\n보 장', 'TEMP_2REPLACE', tmp_text)
        tmp_text = re.sub(r'\n', ' ', tmp_text)
        tmp_text = re.sub(r'TEMP_REPLACE', '\n"보 장', tmp_text)
        tmp_text = re.sub(r'TEMP_2REPLACE', '\n"보 장', tmp_text)
        text_dict[i] = tmp_text.split("\n")
        text += tmp_text

text_list = list(itertools.chain(*text_dict.values()))
chunking_data = [Document(page_content=page, metadata=dict(page=i)) for i, page in enumerate(text_list)]

#### **청킹 된 외부 참조 데이터를 벡터 DB에 저장**

In [39]:
vector_store = FAISS.from_documents(chunking_data, embeddings)

#### **보험 청구인의 사고 사례(예시)**

In [40]:
diagnosis_str = ""

In [41]:
medical_info = defaultdict(list)
medical_info["hospital_1"] = (
    '질병 또는 부상명: (S52590) 요골 하단의 상세불명 골절, 폐쇄성, '
    '(S62630) 기타 손가락의 중지골 골절, 폐쇄성, '
    '(S836) 무릎의 기타 및 상세불명 부분의 염좌 및 긴장, '
    '(M2416) 기타 관절연골장애, 무릎관절, '
    '(M2406) 관절안의 유리체 무릎관절, '
    '(M6586) 기타 윤활막염 및 힘줄윤활막염 무릎관절. '
    '\n치료기간: 입원 2022년 11월 26일부터 2022년 12월 24일까지(29 일간). '
    '\n소견서 내용: 상기환자는 2022/11/1 넘어져 수상후 타병원 진료후 내원하신 환자로 '
    '우측 요골부, 우측 제4 수지부의 골절 진단과 우측 무릎의 통증으로 '
    '안정가료 및 통증완화에 대한 치료를 위하여 상기 기간 동안 입원치료하였으며 '
    '추후 우측 손목, 무릎의 지속적인 관찰 및 재활치료가 필요 할 것으로 사료됨. '
    '\n의료기관 명칭: 사각 종합병원\n\n'
)

medical_info["hospital_2"] = (
    '질병 또는 부상명: (M2416) 기타 관절연골장애 아래다리, '
    '(M2406) 관절안의 유리체, 아래다리, '
    '(M6586) 기타 윤활막염 및 힘줄윤활막염 아래다리, '
    '(M170) 앙쪽 원발성 무릎관절증, '
    '(S52590) 요골 하단의 상세불명 골절, 폐쇄성, '
    '(S62630) 기타 손가락의 중지골 골절, 폐쇄성. '
    '\n치료기간: 입원 2022년 12월 24일부터 2023년 01월 10일까지(18 일간). '
    '\n소견서 내용: 상기환자 양측 무릎과 우측 손목, 우측 네번째 손가락 통증으로 입원한 환자로 '
    '이학적 검사 및 단순 방사선 검사상 상기 병명으로 진단되었으며 '
    'VAS 7의 무릎 통중과 우측 손목 손가락 골절로 인한 ROM 제한으로 증상 호전시끼지 '
    '물리치료, 도수치료, 체외 충격파 치료 등 보존적 치료가 필요할 것으로 판단됨 '
    '상기 소견은 초진 소견이며, 추후 경과에 따라 재평가 요함.'
    '\n구두소견: 사각 병원에서 치료 후에 전원 온 환자로 입원 경위에 대해서는, '
    '골절, 관절 내 유리체, 퇴행성 관절염, 강직 등 단순 통증으로 내원한 것 외에는, '
    '더 이상 드릴 답변 없음. '
    '필요시 더 입원을 할 수도 있는 환자이고 심평원 적정 의료 기준에 따라 퇴원시킨 것으로, '
    '적정입원기간을 명확하게 산정하기 어려움.'
    '\n의료기관 명칭: 서울 창업허브 종합병원\n\n'
)

medical_info["interview_1"] = (
    '고객 안내일자: 2023-02-08.'
    '\n고객 안내내용: 현장심사 안내.'
    '\n고객 반응: 문답서 작성과 면담을 거부함.'
)

medical_info["interview_2"] = (
    '고객 안내일자: 2023-03-17.'
    '\n고객 안내내용: 최초 내원경위로 2022.11.1에 넘어지고 나서, '
    '요골 하단의 상세불명 골절, 폐쇄성 진단으로 주병명이 확인되어, '
    '재해로 검토되어질 수 있음을 안내.'
    '\n고객 반응: 넘어져서 내원한 것은 맞으나, '
    '어깨, 무릎 등은 원래부터 가지고 있는 질병으로 인해 입원치료를 받은 것이니, '
    '질병으로 처리됨이 타당함.'
)

values_list = list(medical_info.values())
result_string = ' '.join(values_list)

#### **RAG가 적용되지 않은 상태에서의 생성 결과**

In [46]:
messages = [
    {"role": "system", "content": "너는 전문성이 매우 높은 수준의 손해사정보고서 작성 챗봇이야. 전문적인 용어로 답변해"},
    {"role": "user", "content": "아래의 조사기록을 분석해서 손해사정보고서 작성" + result_string},
]

result_stream = ollama.chat(
      # model="aya:8b",
      model="qwen2.5:1.5b",
      messages=messages,
      stream=True
    )

In [ ]:
contents_output = result_stream.message.content

In [ ]:
for chunk in result_stream:
    content = chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)

#### **RAG 적용을 위해 벡터 DB에 저장된 텍스트와 유사도 비교**

In [52]:
relevant_docs = vector_store.similarity_search(result_string, k=3)

In [53]:
insurance_context = "\n".join(doc.page_content for doc in relevant_docs)

In [ ]:
insurance_context # 유사도 비교 후 선택된 가장 유사한 정보 3개

In [55]:
input_prompt = (
        f'Context: {insurance_context} \n '
        f'Question: 위의 데이터를 바탕으로 다음의 손해사정보고서에 기록된 질병 또는 부상에 대한 보험금 지급 여부 및 예상금액 판단 \n'
        f' {contents_output}'
    )

In [56]:
messages = [
    {"role": "system", "content": "너는 전문성이 매우 높은 수준의 손해사정보고서 작성 챗봇이야. 전문적인 용어로 답변해"},
    {"role": "user", "content": "아래의 조사기록을 분석해서 손해사정보고서 작성" + result_string},
    {"role": "assistant", "content": contents_output},
    {"role": "user", "content": input_prompt},
]

In [ ]:
result_stream = ollama.chat(
      # model="aya:8b",
      model="qwen2.5:1.5b",
      messages=messages,
      stream=True
    )

for chunk in result_stream:
    content = chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
print('\n')